![CH5-ADS.png](.\Media\CH5-ADS.png)
# <span style="color:#cc5500;">DBCC CHECKDB Internals and Performance</span>

In order for you to gain a more advanced knowledge of working with and correcting SQL Server corruptions, also involves knowing a bit more about how DBCC CHECKDB works and how to enhance its performance.  If you jumped right to this notebook and you have not already mastered the basics of handling corruption, it is recommended that you go through the previous notebooks in this chapter first.

This notebook will cover:

- Various internals of DBCC CHECKDB
- Trace flags to aid DBCC CHECKDB performance
- Parallelism and memory settings for DBCC CHECKDB

## <span style="color:#cc5500;">What exactly does DBCC CHECKDB do?</span>

- The first thing it does is perform Primitive checks of critical system tables
    - The 3 critical system Storage Engine metadata tables it checkes are sys.allocation\_units, sys.sysrowsets and sys.sysrscols
    - Makes sure that the leaf-level clustered index pages of these 3 metadata tables can be read into memory and looks for any improper loops in the linkage
- Allocation checks.  This is the equivalent of DBCC CHECKALLOC
    - Validates the GAM, SGAM, PFS and IAM Pages bit settings are correct and that no two pages have an identical bit set
- Logical checks of critical system tables
- Logical checks of all other tables in the database
- Metadata checks.  This is the Equivalent to DBCC CHECKCATALOG
- Service Broker data validation
- Indexed view, XML index, spatial index checks
    - Only when the WITH EXTENDED\_LOGICAL\_CHECKS option is used, from SQL Server 2008 onward
- If a repair option is specified, repairs are done at each stage if necessary and possible
- Uses the Query Processor extensively

Fact Processing

- DBCC CHECKDB reads all allocated pages in the database in a very efficient manner
- It reads the pages in allocation order, driving its own readahead, and generates “facts” about what it has seen, which is O(n\*log(n))
    - E.g. facts about pages, b-tree linkages, off-row text linkages
    - Driven by an internal structure called the MultiObjectScanner
- Each fact has a multi-part key
    - Including object ID, index ID, allocation unit ID, page ID
- Facts are passed to the Query Processor for efficient sorting and hashing, then passed back to DBCC for aggregation
    - Aggregation means that facts cancel each other out
    - When extra or missing facts are detected, that’s a corruption
      
        
        ![CH5-FactProcessing.jpg](.\Media\CH5-FactProcessing.jpg)
        ![CH5-FactProcessing2.jpg](.\Media\CH5-FactProcessing2.jpg)
        

Batch Processing

- The fact-storage worktable usually requires more memory than is available so it spills into tempdb
- The batch size is limited to prevent excessive tempdbspace usage
    - A batch’s “size” is the number of tables and associated indexes
- The minimum batch size is a single table and its nonclustered indexes as everything about a table must be checked at the same time
- Tables are added to the batch until one of the following occurs:
    - The total number of indexes in the batch becomes more than 512
    - The estimate for all necessary facts for the batch becomes more than 32 MB
    - If a repair option is specified, the batch size is always limited to a single table
- Read and write performance of tempdb can greatly affect the run time

Parallelism

- DBCC CHECKDB will run in parallel on Enterprise Edition
    
    - Also applies to DBCC CHECKTABLE and DBCC CHECKFILEGROUP
    - DBCC CHECKALLOCand DBCC CHECKCATALOG are always single-threaded
- Each batch executes the DBCC CHECKDB internal “query” and the Query Processor decides how far to parallelize
    
    - Up to the limit imposed by the sp\_configure MAXDOP setting or the Resource Governor workload group MAX\_DOP setting
    - There is no option to set DBCC CHECKDB’s degree of parallelism
- Each thread is given a page to generate facts for, and then calls into the MultiObjectScanner to get the next page to process
    
- Threads then participate in fact aggregation once all pages have been read and processed for facts
    
- Parallelism can be disabled using documented trace flag 2528
    
    - Greatly increases run time, but greatly decreases the resources required
    
    ![CH5-Parallelism.jpg](.\Media\CH5-Parallelism.jpg)

cc